In [1]:
import xarray as xr
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings

In [2]:
path='/mnt/1A42C1DD42C1BE2F/MyProjects/ML_HEATWAVE/Specific Humidity Multiple levels/*.nc'
shum=xr.open_mfdataset(path)


In [3]:
shum

<xarray.Dataset>
Dimensions:    (level: 8, lat: 73, lon: 144, time: 15340, nbnds: 2)
Coordinates:
  * level      (level) float32 1e+03 925.0 850.0 700.0 600.0 500.0 400.0 300.0
  * lat        (lat) float32 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * lon        (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Dimensions without coordinates: nbnds
Data variables:
    shum       (time, level, lat, lon) float32 dask.array<chunksize=(365, 8, 73, 144), meta=np.ndarray>
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(365, 2), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          mean daily NMC reanalysis (1981)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    history:        created 95/06/06 by Hoop (netCDF2.3)\nConverted to chunke...
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...

In [4]:
lat_range = slice(29,18.0)  # take lat values in backwards directions.
lon_range = slice(85, 97)  # 88.0 degrees east to 92.7 degrees east
desired_levels=[1000,850,500,300]
# desired_levels=[300]

# Use the sel method to select data within the specified Bangladesh region
bangladesh_shum = shum.sel(lat=lat_range,lon=lon_range,level=desired_levels)

In [5]:
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm# Import tqdm
warnings.filterwarnings('ignore')

# # Open your netCDF dataset
# ds = xr.open_dataset('your_dataset.nc')

# Define the chunk size
chunk_size = 500  # Adjust this based on your available memory

# Initialize an empty DataFrame to store the results
shum_df = pd.DataFrame(columns=['time', 'lat', 'lon', 'shum_1000hpa','shum_850hpa','shum_500hpa','shum_300hpa'])

# Calculate the number of chunks
num_chunks = len(bangladesh_shum.time)

# Create a tqdm progress bar
for start in tqdm(range(0, num_chunks, chunk_size),desc='Main Loop'):
    end = start + chunk_size
    chunk = bangladesh_shum.isel(time=slice(start, end))

    # Extract the data
    time = chunk.time.values
    lat = chunk.lat.values
    lon = chunk.lon.values
    shum_1000hpa = chunk.sel(level=1000).shum.values
    shum_850hpa = chunk.sel(level=850).shum.values
    shum_500hpa = chunk.sel(level=500).shum.values
    shum_300hpa = chunk.sel(level=300).shum.values
    

    # Create a DataFrame for this chunk and append it to the main DataFrame
    chunk_df = pd.DataFrame({
        'time': time.repeat(len(lat) * len(lon)),
        'lat': np.tile(lat, len(time) * len(lon)),
        'lon': np.tile(lon, len(time) * len(lat)),
        'shum_1000hpa': shum_1000hpa.ravel(),
        'shum_850hpa': shum_850hpa.ravel(),
        'shum_500hpa': shum_500hpa.ravel(),
        'shum_300hpa': shum_300hpa.ravel()
    })

    shum_df = shum_df.append(chunk_df, ignore_index=True)

# t2m_df now contains the concatenated data
shum_df

Main Loop:   0%|          | 0/31 [00:00<?, ?it/s]

,time,lat,lon,shum_1000hpa,shum_850hpa,shum_500hpa,shum_300hpa
0,1981-01-01,27.5,85.0,0.007865,0.005078,0.000320,0.000132
1,1981-01-01,25.0,87.5,0.008783,0.005775,0.000120,0.000063
2,1981-01-01,22.5,90.0,0.009350,0.006583,0.000249,0.000033
3,1981-01-01,20.0,92.5,0.010030,0.007505,0.000610,0.000022
4,1981-01-01,27.5,95.0,0.011783,0.008733,0.000667,0.000031
...,...,...,...,...,...,...,...
306795,2022-12-31,20.0,85.0,0.010760,0.006025,0.000693,0.000120
306796,2022-12-31,27.5,87.5,0.010610,0.004300,0.000657,0.000133
306797,2022-12-31,25.0,90.0,0.008787,0.002942,0.000577,0.000148
306798,2022-12-31,22.5,92.5,0.008038,0.002800,0.000610,0.000142
